In [1]:
from haystack import Document
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models as qmodels
import json
import glob
import os
import subprocess
import requests

from config import DATA_DIR, COLLECTION_NAME, EMBEDDER_MODEL_NAME, QDRANT_HOST, QDRANT_PORT, QDRANT_DATA_FOLDER

/home/hpougy/projects/chatbot_156/venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
QDRANT_DATA_FOLDER = os.path.join(os.getcwd(), QDRANT_DATA_FOLDER)
if not os.path.exists(QDRANT_DATA_FOLDER):
    os.makedirs(QDRANT_DATA_FOLDER)

In [3]:
import subprocess
import os

qdrant_dir = os.path.join(os.getcwd(), "qdrant_data")

subprocess.run(
    ["docker", "stop", "qdrant-local"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

subprocess.run(
    ["docker", "rm", "qdrant-local"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)


process_output = subprocess.run([
    "docker", "run", "-d",
    "--name", "qdrant-local",
    "-p", f"{QDRANT_PORT}:6333",
    "-v", f"{QDRANT_DATA_FOLDER}:/qdrant/storage",
    "qdrant/qdrant",
    ],  capture_output=True, text=True)
print("Container id:", process_output.stdout)

Container id: 62dbfb9815e5b5f7d4ad299bdd57bd245311e34ddde3256ca3b3023425f8739a



In [4]:
import time
time.sleep(5)
#test connection to qdrant
with requests.get("http://localhost:6333/collections")as r:
    r_qdrant = r.json()
    assert r_qdrant['status'] == 'ok'

In [5]:

qdrant_client = QdrantClient(host=QDRANT_HOST, port=QDRANT_PORT)

In [6]:
CHUNK_JSON = os.path.join(DATA_DIR, "chunks_metadata.json")

with open(CHUNK_JSON, "r", encoding="utf-8") as f:
    chunks_json = json.load(f)


In [7]:
chunks_json['chunks'][0]


'[DT_ATUALIZACAO | Adotar cães e gatos]\n20/10/2025'

In [8]:
chunks_json['metadata'][0]


{'service_name': 'Adotar cães e gatos',
 'service_id': 3676,
 'theme': 'Animais',
 'subtheme': 'Adoção de animais'}

In [9]:
docs = [Document(content=chunk, meta=meta) for chunk, meta in zip(chunks_json['chunks'], chunks_json['metadata'])]

In [10]:
len(docs)==len(chunks_json['chunks'])

True

In [11]:
embedder = SentenceTransformer(EMBEDDER_MODEL_NAME, device="cuda")


texts = [d.content for d in docs]

vectors = embedder.encode(texts, 
                            normalize_embeddings=True,
                            show_progress_bar=True,
                            batch_size=4,        # mudar se estiver estourar a memoria da GPU
                            )

len(vectors), len(vectors[0])  # deve mostrar (n_chunks, 1024)


Batches: 100%|██████████| 5967/5967 [04:48<00:00, 20.65it/s]


(17901, 1024)

In [ ]:
vector_dim = embedder.get_sentence_embedding_dimension()

qdrant_client.recreate_collection(
    collection_name="servicos_156",
    vectors_config=qmodels.VectorParams(
        size=vector_dim,
        distance=qmodels.Distance.COSINE
    )
)
 
print("Coleção criada.")

/tmp/ipykernel_44447/964748076.py:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


Coleção criada.


In [13]:
payloads = [d.meta for d in docs]

qdrant_client.upload_collection(
    collection_name="servicos_156",
    vectors=vectors,
    payload=payloads,
    ids=None,        # IDs automáticos
    batch_size=64
)

print("Embeddings enviados com sucesso!")


Embeddings enviados com sucesso!


In [32]:
query = "quais documentos preciso para adotar um gato?"

query_vec = embedder.encode(query, normalize_embeddings=True).tolist()

results = qdrant_client.query_points(
    collection_name="servicos_156",
    query=query_vec,
    limit=5
)

results


QueryResponse(points=[ScoredPoint(id='53ca5064-dd5e-4a5e-b97b-6ddb72eb3ef6', version=1, score=0.7425673, payload={'service_name': 'Adotar cães e gatos', 'service_id': 3676, 'theme': 'Animais', 'subtheme': 'Adoção de animais'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id='0e3dae52-14d0-4fe0-9fe5-c30c3fac6aec', version=1, score=0.6735799, payload={'service_name': 'Adotar cães e gatos', 'service_id': 3676, 'theme': 'Animais', 'subtheme': 'Adoção de animais'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id='1e7947e4-f668-4ebe-b9b7-28f6eedf2c5a', version=1, score=0.6728593, payload={'service_name': 'Adotar cães e gatos', 'service_id': 3676, 'theme': 'Animais', 'subtheme': 'Adoção de animais'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id='14a0e529-87b2-4500-bb7e-8893b2db7cb6', version=1, score=0.6611711, payload={'service_name': 'Adotar cães e gatos', 'service_id': 3676, 'theme': 'Animais', 'subtheme': 'Adoção de animais'}, vector=None, s

In [33]:
results.model_dump()

{'points': [{'id': '53ca5064-dd5e-4a5e-b97b-6ddb72eb3ef6',
   'version': 1,
   'score': 0.7425673,
   'payload': {'service_name': 'Adotar cães e gatos',
    'service_id': 3676,
    'theme': 'Animais',
    'subtheme': 'Adoção de animais'},
   'vector': None,
   'shard_key': None,
   'order_value': None},
  {'id': '0e3dae52-14d0-4fe0-9fe5-c30c3fac6aec',
   'version': 1,
   'score': 0.6735799,
   'payload': {'service_name': 'Adotar cães e gatos',
    'service_id': 3676,
    'theme': 'Animais',
    'subtheme': 'Adoção de animais'},
   'vector': None,
   'shard_key': None,
   'order_value': None},
  {'id': '1e7947e4-f668-4ebe-b9b7-28f6eedf2c5a',
   'version': 1,
   'score': 0.6728593,
   'payload': {'service_name': 'Adotar cães e gatos',
    'service_id': 3676,
    'theme': 'Animais',
    'subtheme': 'Adoção de animais'},
   'vector': None,
   'shard_key': None,
   'order_value': None},
  {'id': '14a0e529-87b2-4500-bb7e-8893b2db7cb6',
   'version': 1,
   'score': 0.6611711,
   'payload': {'

In [37]:
#from config import GEN_MODEL_NAME

def ollama_running():
    try:
        subprocess.run(["pgrep", "-f", "ollama"], check=True, stdout=subprocess.DEVNULL)
        return True
    except subprocess.CalledProcessError:
        return False

ollama_is_running =  ollama_running()

if not ollama_is_running:
    print("Ollama não está em execução. Inicie o Ollama para usar o modelo de geração de texto.")
        # Launch Ollama serve in background
    ollama_process = subprocess.Popen(
        ["ollama", "serve"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )

    print("Ollama server iniciado.")


#baixando o modelo
subprocess.run(["ollama", "pull", 'qwen2.5:7b'])

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 2bada8a74506:   0% ▕                  ▏ 258 KB/4.7 GB                  pulling manifest 
pulling 2bada8a74506:   0% ▕                  ▏ 7.3 MB/4.7 GB                  pulling manifest 
pulling 2bada8a74506:   0% ▕                  ▏  12 MB/4.7 GB                  pulling manifest 
pulling 2bada8a74506:   0% ▕                  ▏  21 MB/4.7 GB                  pulling manifest 
pulling 2bada8a74506:   1% ▕                  ▏  30 MB/4.7 GB                  pulling manifest 
pulling 2bada8a74506:   1% ▕                  ▏  36 MB/4.7 GB                  pulling manifest 
pulling 2bada8a74506:   1% ▕                  ▏  46 MB/4.7 GB                  pulling manifest 
pulling 2bada8a74506:   1% ▕                  ▏  56 MB/4.7 GB                  pulling manifes

CompletedProcess(args=['ollama', 'pull', 'qwen2.5:7b'], returncode=0)

In [65]:
def generate_llm(prompt, model="qwen2.5:7b"):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }

    response = requests.post(url, json=payload)
    response.raise_for_status()

    data = response.json()
    return data.get("response", "")


In [50]:
def retrieve_top_k(qdrant, embedder, query, k=5):
    query_vec = embedder.encode(query, normalize_embeddings=True).tolist()

    results = qdrant.query_points(
        collection_name="servicos_156",
        query=query_vec,
        limit=k,
        with_vectors=False,
        with_payload=True
    )

    # Retornar payloads + score
    return [{
        "score": p.score,
        "metadata": p.payload
    } for p in results.points]


In [51]:
def build_context(points):
    blocks = []
    for p in points:
        meta = p["metadata"]
        block = ""

        if "content" in meta:
            block += f"{meta['content']}\n"
        else:
            # fallback — útil caso tenha esquecido de enviar conteúdo no payload
            for key, value in meta.items():
                block += f"{key}: {value}\n"

        blocks.append(block)

    return "\n---\n".join(blocks)


In [52]:
def build_prompt(query, context):
    return f"""
Você é um assistente do Portal 156 da Prefeitura de São Paulo.
Use APENAS as informações do CONTEXTO abaixo para responder.

Regras:
- Seja direto e objetivo.
- Sem invencionices: use SOMENTE o que estiver no contexto.
- Se faltar informação, diga isso explicitamente.
- Seja claro para o cidadão.

PERGUNTA:
{query}

CONTEXTO:
{context}

RESPOSTA:
"""


In [72]:
def rag_answer(qdrant, embedder, query):
    # 1. Retrieve
    points = retrieve_top_k(qdrant, embedder, query, k=10)
    print(points)

    # 2. Contexto
    context = build_context(points)
    print(context)

    # 3. Prompt
    prompt = build_prompt(query, context)
    print(prompt)

    # 4. Geração via Qwen (notebook)
    resposta = generate_llm(prompt)

    return resposta


In [73]:
query = "para adotar um gato, preciso levar caixa de transporte?"
print(rag_answer(qdrant_client, embedder, query))


[{'score': 0.7581776, 'metadata': {'service_name': 'Adotar cães e gatos', 'service_id': 3676, 'theme': 'Animais', 'subtheme': 'Adoção de animais'}}, {'score': 0.65206313, 'metadata': {'service_name': 'Adotar cães e gatos', 'service_id': 3676, 'theme': 'Animais', 'subtheme': 'Adoção de animais'}}, {'score': 0.60338485, 'metadata': {'service_name': 'Adotar cães e gatos', 'service_id': 3676, 'theme': 'Animais', 'subtheme': 'Adoção de animais'}}, {'score': 0.59358346, 'metadata': {'service_name': 'Adotar cães e gatos', 'service_id': 3676, 'theme': 'Animais', 'subtheme': 'Adoção de animais'}}, {'score': 0.5861659, 'metadata': {'service_name': 'Adotar cães e gatos', 'service_id': 3676, 'theme': 'Animais', 'subtheme': 'Adoção de animais'}}, {'score': 0.5797809, 'metadata': {'service_name': 'Adotar cães e gatos', 'service_id': 3676, 'theme': 'Animais', 'subtheme': 'Adoção de animais'}}, {'score': 0.57427573, 'metadata': {'service_name': 'Adotar cães e gatos', 'service_id': 3676, 'theme': 'Anim